In [ ]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent, PositionsTracker
from qubx.pandaz.utils import *

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.trackers import PortfolioRebalancerTracker
from qubx.core.series import Quote
from qubx.ta.indicators import sma, ema
from qubx.data.readers import CsvStorageDataReader, MultiQdbConnector, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC, AsPandasFrame, RestoreTicksFromOHLC, AsOhlcvSeries
from qubx.core.basics import Deal, Instrument, Order, Position, Signal
from qubx.core.utils import recognize_time

from qubx.impl.ccxt_connector import CCXTExchangesConnector
from qubx.impl.ccxt_trading import CCXTTradingConnector
from qubx.utils.runner import get_account_config

def Q(time: str, bid: float, ask: float) -> Quote:
    return Quote(recognize_time(time), bid, ask, 0, 0)

# r = CsvStorageDataReader("../tests/data/csv")
# r = MultiQdbConnector('xlydian-data')

from qubx import QubxLogConfig
from qubx.backtester.simulator import simulate

# from qubx.core.metrics import tearsheet, portfolio_metrics

# Strat for signals testing

In [ ]:
class StrategyForTracking(IStrategy):
    timeframe: str = "1Min"
    fast_period = 5
    slow_period = 12

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> List[Signal] | None:
        for i in ctx.instruments:
            # logger.info(f"\t{i.symbol} : {ctx.broker_provider.get_quote(i.symbol)}")
            ohlc = ctx.ohlc(i, self.timeframe)
            fast = ema(ohlc.close, self.fast_period)
            slow = ema(ohlc.close, self.slow_period)
            pos = ctx.positions[i.symbol].quantity

            if pos <= 0:
                if (fast[0] > slow[0]) and (fast[1] < slow[1]):
                    ctx.trade(i, abs(pos) + i.min_size * 10)

            if pos >= 0:
                if (fast[0] < slow[0]) and (fast[1] > slow[1]):
                    ctx.trade(i, -pos - i.min_size * 10)

        return None

    def ohlcs(self, timeframe: str) -> Dict[str, pd.DataFrame]:
        return {s.symbol: self.ctx.ohlc(s, timeframe).pd() for s in self.ctx.instruments}

    def tracker(self, ctx: StrategyContext) -> PositionsTracker:
        return PortfolioRebalancerTracker(1000, 0).set_context(ctx)